### [Gradient Boosting Decision Tree (GBDT) Regression Model](https://medium.com/top-python-libraries/python-implementation-of-gradient-boosting-decision-tree-gbdt-regression-model-with-residual-04d0174a46a5)

In [1]:
!pip install -q -U joblib shap PyALE

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.4/308.4 kB 6.7 MB/s eta 0:00:00


In [2]:
import os
import time
import shap
import joblib
import warnings
import matplotlib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from PyALE import ale

from itertools import combinations
from collections import defaultdict

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.inspection import PartialDependenceDisplay, partial_dependence

from scipy.stats import gaussian_kde

In [ ]:
# --- Global Settings ---
warnings.filterwarnings("ignore", category=FutureWarning, module="sklearn.utils._bunch")
warnings.filterwarnings("ignore", category=UserWarning)  # Ignore known data type mismatch warnings
matplotlib.use('TkAgg')  # It is recommended to set this at the top of the script to avoid backend conflicts

print('-------------------------------------Preparing Data---------------------------------------')
# Please ensure the file path is correct
df = pd.read_excel(r'D:\data.xlsx')

# --- Data Processing Correction: Use Pandas format throughout to retain feature names ---
y = df.iloc[:, 0]
x = df.iloc[:, 1:]
feature_names_from_df = x.columns.tolist()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train_scaled_df = pd.DataFrame(scaler.fit_transform(x_train), columns=x_train.columns, index=x_train.index)
X_test_scaled_df = pd.DataFrame(scaler.transform(x_test), columns=x_test.columns, index=x_test.index)

In [ ]:
print('-------------------------------------Defining GBDT model hyperparameter range---------------------------------------')

n_estimators_range = range(100, 301, 100)
max_depth_range = range(3, 7, 1)
learning_rate_range = [0.01, 0.1, 0.2]

print('-------------------------------------Searching for the best hyperparameters---------------------------------------')

gbdt_param_grid = {
    'n_estimators': n_estimators_range,
    'max_depth': max_depth_range,
    'learning_rate': learning_rate_range
}

gd = GridSearchCV(estimator=GradientBoostingRegressor(random_state=0),
                  param_grid=gbdt_param_grid,
                  cv=5,
                  n_jobs=-1,
                  verbose=0)

gd.fit(X_train_scaled_df, y_train)

print('-------------------------------------Outputting the best model---------------------------------------')
print("Best result validated in cross-validation:", gd.best_score_)
print("Best parameter model:", gd.best_estimator_)
print("(dict) Best parameters:", gd.best_params_)

print('-------------------------------------Saving the best model---------------------------------------')

model_save_dir = r'D:\folder'

os.makedirs(model_save_dir, exist_ok=True)
model_path = os.path.join(model_save_dir, 'GBDT_model_final.joblib')
joblib.dump(gd.best_estimator_, model_path)
print(f"Model has been saved to: {model_path}")
loaded_model = joblib.load(model_path)

In [ ]:
print('-------------------------------Apply the model--------------------------------------')
y_test_pred = loaded_model.predict(X_test_scaled_df)
y_train_pred = loaded_model.predict(X_train_scaled_df)

print('-------------------------------------Train the model---------------------------------------')
train_mse = mean_squared_error(y_train, y_train_pred)
train_rmse = np.sqrt(train_mse)
train_mae = mean_absolute_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)
train_n = len(X_train_scaled_df)
print(f'MSE: {train_mse:.4f}, RMSE: {train_rmse:.4f}, MAE: {train_mae:.4f}, R2: {train_r2:.4f}, N: {train_n}')

print('-------------------------------------Evaluate the model performance---------------------------------------')
test_mse = mean_squared_error(y_test, y_test_pred)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)
test_n = len(X_test_scaled_df)
print(f'MSE: {test_mse:.4f}, RMSE: {test_rmse:.4f}, MAE: {test_mae:.4f}, R2: {test_r2:.4f}, N: {test_n}')

print('----------------------------------------Plot the reuslt-----------------------------------------')
results_plot_save_dir = r'D:\foler'


def plot_regression_fit(y_true, y_pred, r2, rmse, mae, data_label_en, title_en, save_path):
    plt.style.use('seaborn-v0_8-whitegrid')
    plt.rcParams['font.family'] = 'sans-serif'
    fig, ax = plt.subplots(figsize=(7, 7))
    ax.scatter(y_true, y_pred, alpha=0.6, edgecolors='k', label=f'{data_label_en} (n={len(y_true)})')
    lims = [np.min([y_true.min(), y_pred.min()]) - 5, np.max([y_true.max(), y_pred.max()]) + 5]
    ax.plot(lims, lims, 'k--', alpha=0.75, zorder=0, label='1:1 Line (Perfect Fit)')
    ax.set_aspect('equal')
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    # Ensure y_true and y_pred are numpy arrays for polynomial fitting
    y_true_np = np.array(y_true)
    y_pred_np = np.array(y_pred)
    m, b = np.polyfit(y_true_np, y_pred_np, 1)
    ax.plot(y_true_np, m * y_true_np + b, 'r-', label='Linear Fit')
    ax.set_xlabel('True Values (%)', fontsize=12)
    ax.set_ylabel('Predicted Values (%)', fontsize=12)
    ax.set_title(title_en, fontsize=14, weight='bold')
    metrics_text = f'$R^2 = {r2:.3f}$\n$RMSE = {rmse:.3f}$\n$MAE = {mae:.3f}$'
    ax.text(0.05, 0.95, metrics_text, transform=ax.transAxes, fontsize=12, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    ax.legend(loc='lower right')
    fig.savefig(save_path, dpi=200, bbox_inches='tight')
    plt.close(fig)


train_path = os.path.join(results_plot_save_dir, 'GBDT_TrainingSet_final.png')
test_path = os.path.join(results_plot_save_dir, 'GBDT_TestSet_final.png')
plot_regression_fit(y_train, y_train_pred, train_r2, train_rmse, train_mae, 'Train Set',
                    'GBDT Model Performance (Train Set)', train_path)
plot_regression_fit(y_test, y_test_pred, test_r2, test_rmse, test_mae, 'Test Set', 'GBDT Model Performance (Test Set)',
                    test_path)

plt.rcdefaults()

In [ ]:
def plot_importance_combined(df_importance, title, save_path, bar_color='dodgerblue'):
    """
    A general-purpose function for plotting a combination chart of a
    'horizontal bar plot + inset donut chart'.

    Parameters:
    - df_importance: DataFrame containing 'Feature' and 'Importance' columns.
    - title: The main title of the chart.
    - save_path: The file path to save the image.
    - bar_color: The color of the bars in the bar chart.
    """
    # To plot a horizontal bar chart with the most important feature at the top,
    # sort the DataFrame by importance in ascending order.
    df_importance_sorted = df_importance.sort_values(by='Importance', ascending=True)


    plt.rc("font", family='sans-serif')

    # Create the main figure and axes.
    fig, ax = plt.subplots(figsize=(14, 10))

    # Plot the horizontal bar chart.
    bars = ax.barh(df_importance_sorted['Feature'], df_importance_sorted['Importance'], color=bar_color, alpha=0.8)
    ax.set_title(title, fontsize=18, pad=20)
    ax.set_xlabel('Importance Score', fontsize=14)
    ax.set_ylabel('Feature', fontsize=14)
    ax.tick_params(axis='both', which='major', labelsize=12)
    ax.grid(axis='x', linestyle='--', alpha=0.6)

    # Add value labels to the bars.
    for bar in bars:
        width = bar.get_width()
        ax.text(width, bar.get_y() + bar.get_height() / 2,
                f' {width:.4f}', va='center', ha='left', fontsize=10)

    # Adjust the X-axis range to make space for the labels.
    ax.set_xlim(right=ax.get_xlim()[1] * 1.2)

    # --- Plot the inset donut chart ---
    N_DONUT_FEATURES = 5
    if len(df_importance) < N_DONUT_FEATURES:
        N_DONUT_FEATURES = len(df_importance)

    # Sort in descending order to get the top N features.
    df_desc = df_importance.sort_values(by='Importance', ascending=False)
    top_n_features = df_desc.head(N_DONUT_FEATURES)
    donut_feature_names = top_n_features['Feature'].tolist()

    # Proceed only if there are features with importance greater than 0.
    if not top_n_features.empty and top_n_features['Importance'].sum() > 0:
        # Calculate percentages for the donut chart slices.
        total_donut_importance = top_n_features['Importance'].sum()
        donut_percentages = top_n_features['Importance'] / total_donut_importance * 100

        # Add an inset axis for the donut chart.
        ax_inset = fig.add_axes([0.45, 0.15, 0.28, 0.28]) # [left, bottom, width, height]

        colors = matplotlib.colormaps['tab10'].colors
        wedges, _ = ax_inset.pie(
            donut_percentages,
            colors=colors[:len(top_n_features)], startangle=90, counterclock=False,
            wedgeprops=dict(width=0.45, edgecolor='w') # The 'width' property creates the donut hole.
        )

        # Add text to the center of the donut chart.
        subset_importance_ratio = top_n_features['Importance'].sum() / df_importance['Importance'].sum()
        ax_inset.text(0, 0, f'Top {N_DONUT_FEATURES} Features\nAccount for\n{subset_importance_ratio:.2%} of Total Importance',
                      ha='center', va='center', fontsize=9, linespacing=1.4)

        # Logic to add and position labels for the donut slices.
        label_threshold = 3.0
        y_text_offsets = {'left': 1.4, 'right': 1.4}
        for i, p in enumerate(wedges):
            percent = donut_percentages.iloc[i]
            ang = (p.theta2 - p.theta1) / 2. + p.theta1
            y = np.sin(np.deg2rad(ang))
            x = np.cos(np.deg2rad(ang))

            # For small slices, draw a line to an external label.
            if percent < label_threshold and percent > 0:
                side = 'right' if x > 0 else 'left'
                y_pos = y_text_offsets[side]
                y_text_offsets[side] += -0.2 if y > 0 else 0.2
                connectionstyle = f"angle,angleA=0,angleB={ang}"
                ax_inset.annotate(f'{percent:.1f}%', xy=(x, y), xytext=(1.2 * np.sign(x), y_pos),
                                  fontsize=9, ha='center',
                                  arrowprops=dict(arrowstyle="-", connectionstyle=connectionstyle, relpos=(0.5, 0.5)))
            # For larger slices, place the label inside.
            elif percent > 0:
                ax_inset.text(x * 0.8, y * 0.8, f'{percent:.1f}%', ha='center', va='center', fontsize=9,
                              fontweight='bold', color='white')

        # Add a legend for the donut chart outside the pie area.
        ax_inset.legend(wedges, donut_feature_names,
                        loc="center left", bbox_to_anchor=(1.2, 0.5),
                        frameon=False, fontsize=11)

    # Save the figure to the specified path with high resolution.
    plt.savefig(save_path, dpi=720, bbox_inches='tight')
    plt.close(fig)

In [ ]:
print('----------------------------------------Calculating and Plotting GBDT Native Feature Importance-----------------------------------------')
importances = loaded_model.feature_importances_
gbdt_importance_df = pd.DataFrame({'Feature': feature_names_from_df, 'Importance': importances})
save_path_gbdt = os.path.join(results_plot_save_dir, 'GBDT_Feature_Importance_Combined_Plot_final.png')
plot_importance_combined(gbdt_importance_df, 'Feature Importance Calculated by GBDT Model', save_path_gbdt, bar_color='dodgerblue')

In [ ]:
# --- 2. Permutation Importance Plot ---
print("----------------------------------------Calculating and plotting Permutation Importance-----------------------------------------")
scores = defaultdict(list)
for feat_name in feature_names_from_df:
    X_t = X_test_scaled_df.copy()
    X_t[feat_name] = np.random.permutation(X_t[feat_name].values)
    shuff_acc = r2_score(y_test, loaded_model.predict(X_t))
    scores[feat_name].append((test_r2 - shuff_acc) / test_r2 if test_r2 > 1e-6 else test_r2 - shuff_acc)

sorted_scores = sorted([(np.mean(score_list), feat) for feat, score_list in scores.items()], reverse=True)

perm_feature_names = [feat for _, feat in sorted_scores]
perm_feature_scores = [score for score, _ in sorted_scores]

perm_importance_df = pd.DataFrame({'Feature': perm_feature_names, 'Importance': perm_feature_scores})

save_path_perm = os.path.join(results_plot_save_dir, 'GBDT_Feature_Importance_Permutation_final.png')

plot_importance_combined(perm_importance_df, 'Feature Importance (Permutation Importance for GBDT)', save_path_perm, bar_color='lightcoral')

In [ ]:
print('----------------------------------------Plotting Residual Analysis-----------------------------------------')
train_residuals = y_train - y_train_pred
test_residuals = y_test - y_test_pred

def plot_residuals_styled(residuals, y_pred, save_path, title):
    plt.style.use('seaborn-v0_8-whitegrid')
    plt.rc("font", family='Arial')
    fig, ax = plt.subplots(figsize=(10, 8))
    sd_residuals = np.std(residuals)
    is_outlier = np.abs(residuals) > 2 * sd_residuals
    num_outliers = np.sum(is_outlier)
    print(f"In dataset '{title}', found {num_outliers} outliers (residual > 2 S.D.).")
    sd_label = f'S.D. (±{sd_residuals:.2f})'
    ax.axhspan(-sd_residuals, sd_residuals, color='yellow', alpha=0.3, label=sd_label)
    ax.scatter(y_pred[~is_outlier], residuals[~is_outlier], alpha=0.6, c='green',
               edgecolors='k', linewidth=0.5, s=50, label='Normal')
    ax.scatter(y_pred[is_outlier], residuals[is_outlier], alpha=0.8, c='red',
               edgecolors='k', linewidth=0.5, s=70, label='Outlier (> 2 S.D.)')
    ax.axhline(0, color='black', linestyle='--', linewidth=1.5)
    ax.set_title(title, fontsize=16, weight='bold')
    ax.set_xlabel('Predicted Value', fontsize=14)
    ax.set_ylabel('Residual (True - Predicted)', fontsize=14)
    for spine in ax.spines.values():
        spine.set_visible(True)
        spine.set_color('black')
        spine.set_linewidth(1)
    ax.legend(loc='upper right', fontsize=12)
    y_max = np.max(np.abs(residuals)) * 1.2
    ax.set_ylim(-y_max, y_max)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

train_res_path = os.path.join(results_plot_save_dir, 'GBDT_train_residual_analysis_final.png')
test_res_path = os.path.join(results_plot_save_dir, 'GBDT_validation_residual_analysis_final.png')
plot_residuals_styled(train_residuals, y_train_pred, train_res_path, 'GBDT Train Residual Analysis')
plot_residuals_styled(test_residuals, y_test_pred, test_res_path, 'GBDT Validation Residual Analysis')

In [ ]:
# --- Complete Section for PDP and ICE Plotting ---
print('------------------------ Starting PDP and ICE Plotting ------------------------')
pdp_ice_save_dir = os.path.join(results_plot_save_dir, 'GBDT_PDP_ICE_Plots_final')
os.makedirs(pdp_ice_save_dir, exist_ok=True)
pdp_2way_save_dir = os.path.join(pdp_ice_save_dir, '2Way_PDP_All_Combinations')
os.makedirs(pdp_2way_save_dir, exist_ok=True)
pdp_3d_save_dir = os.path.join(pdp_ice_save_dir, '3D_PDP_All_Combinations')
os.makedirs(pdp_3d_save_dir, exist_ok=True)

n_top_features_for_pdp = 6
if n_top_features_for_pdp > len(feature_names_from_df):
    n_top_features_for_pdp = len(feature_names_from_df)
top_features_pdp_names = gbdt_importance_df['Feature'].tolist()[:n_top_features_for_pdp]
plt.style.use('seaborn-v0_8-whitegrid')
plt.rc("font", family='Arial')

print("\nStart plotting univariate PDP (individual)...")
for feature_name_str in top_features_pdp_names:
    try:
        fig_pdp, ax_pdp = plt.subplots(figsize=(8, 6))
        PartialDependenceDisplay.from_estimator(loaded_model, X_train_scaled_df, [feature_name_str], kind='average',
                                                ax=ax_pdp)
        ax_pdp.set_title(f'Partial Dependence Plot (PDP)\nFeature: {feature_name_str}', fontsize=14)
        ax_pdp.set_xlabel(f'{feature_name_str} (Standardized Value)', fontsize=12)
        ax_pdp.set_ylabel('Dependence on Predicted Value (PDP)', fontsize=12)
        plt.savefig(os.path.join(pdp_ice_save_dir, f'GBDT_PDP_single_{feature_name_str}.png'), dpi=300,
                    bbox_inches='tight')
        plt.close(fig_pdp)
    except Exception as e:
        print(f"Error plotting PDP for {feature_name_str}: {e}")

# --- Method 2: Manually compute PDP with shaded 95% confidence interval ---
print("\nStart plotting univariate PDP (with shaded confidence interval)...")
for feature_name_str in top_features_pdp_names:
    try:
        # 1. Use partial_dependence to compute ICE values for each sample
        pd_results = partial_dependence(
            loaded_model,
            X_train_scaled_df,
            [feature_name_str],
            kind='individual'  # must be 'individual' to retrieve each sample's ICE data
        )

        # 2. Compute average (PDP) and standard deviation
        ice_lines = pd_results['individual'][0]
        pdp_line = np.mean(ice_lines, axis=0)
        pdp_std = np.std(ice_lines, axis=0)

        # 3. Get feature value grid
        grid_values = pd_results['values'][0]

        # 4. Plot
        fig, ax = plt.subplots(figsize=(8, 6))

        # PDP average effect line
        ax.plot(grid_values, pdp_line, color='red', linestyle='--', linewidth=2, label='Average Effect (PDP)')

        # 95% Confidence Interval (mean ± 1.96 * std)
        ax.fill_between(
            grid_values,
            pdp_line - 1.96 * pdp_std,
            pdp_line + 1.96 * pdp_std,
            color='skyblue',
            alpha=0.3,
            label='95% Confidence Interval'
        )

        ax.set_title(f'Partial Dependence Plot (PDP) with 95% CI\nFeature: {feature_name_str}', fontsize=14)
        ax.set_xlabel(f'{feature_name_str} (Standardized Value)', fontsize=12)
        ax.set_ylabel('Partial Dependence on Predicted Value', fontsize=12)
        ax.legend()

        save_path = os.path.join(pdp_ice_save_dir, f'GBDT_PDP_with_Shaded_CI_{feature_name_str}.png')
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close(fig)
    except Exception as e:
        print(f"Error plotting PDP (with shaded CI) for {feature_name_str}: {e}")

print("\nStart plotting univariate ICE (individual)...")
for feature_name_str in top_features_pdp_names:
    try:
        fig_ice, ax_ice = plt.subplots(figsize=(8, 6))
        PartialDependenceDisplay.from_estimator(loaded_model, X_train_scaled_df, [feature_name_str], kind='individual',
                                                centered=True, ice_lines_kw={"color": "tab:blue", "alpha": 0.1},
                                                pd_line_kw={'color': 'red', 'linestyle': '--', 'linewidth': 2},
                                                ax=ax_ice)
        ax_ice.set_title(f'Individual Conditional Expectation (ICE)\nFeature: {feature_name_str}', fontsize=14)
        ax_ice.set_xlabel(f'{feature_name_str} (Standardized Value)', fontsize=12)
        ax_ice.set_ylabel('Dependence on Predicted Value (ICE)', fontsize=12)
        plt.savefig(os.path.join(pdp_ice_save_dir, f'GBDT_ICE_single_{feature_name_str}.png'), dpi=300,
                    bbox_inches='tight')
        plt.close(fig_ice)
    except Exception as e:
        print(f"Error plotting ICE for {feature_name_str}: {e}")

print("\nStart plotting combined PDP and ICE...")
for feature_name_str in top_features_pdp_names:
    try:
        fig_comb, ax_comb = plt.subplots(figsize=(8, 6))
        PartialDependenceDisplay.from_estimator(loaded_model, X_train_scaled_df, [feature_name_str], kind='both',
                                                centered=True,
                                                pd_line_kw={"color": "tab:red", "linestyle": "--", "linewidth": 2.5},
                                                ice_lines_kw={"color": "tab:blue", "alpha": 0.1}, ax=ax_comb)
        ax_comb.set_title(f'Combined PDP and ICE\nFeature: {feature_name_str}', fontsize=14)
        ax_comb.set_xlabel(f'{feature_name_str} (Standardized Value)', fontsize=12)
        ax_comb.set_ylabel('Dependence on Predicted Value', fontsize=12)
        plt.savefig(os.path.join(pdp_ice_save_dir, f'GBDT_PDP_ICE_combined_{feature_name_str}.png'), dpi=300,
                    bbox_inches='tight')
        plt.close(fig_comb)
    except Exception as e:
        print(f"Error plotting PDP/ICE for {feature_name_str}: {e}")

print("\nStart plotting bivariate (2D) PDP...")
if len(top_features_pdp_names) >= 2:
    for feat1_name, feat2_name in combinations(top_features_pdp_names, 2):
        try:
            fig_2d_pdp, ax_2d_pdp = plt.subplots(figsize=(8, 7))
            PartialDependenceDisplay.from_estimator(loaded_model, X_train_scaled_df, [(feat1_name, feat2_name)],
                                                    ax=ax_2d_pdp)
            ax_2d_pdp.set_title(f'2D PDP: {feat1_name} vs {feat2_name}', fontsize=16)
            plt.tight_layout()
            plt.savefig(os.path.join(pdp_2way_save_dir, f'GBDT_PDP_2D_{feat1_name}_{feat2_name}.png'), dpi=300)
            plt.close(fig_2d_pdp)
        except Exception as e:
            print(f"Error plotting 2D PDP for {feat1_name} & {feat2_name}: {e}")

# --- [Fixed] Section for 3D PDP Plotting ---
print("\nStart plotting bivariate (3D) PDP...")
if len(top_features_pdp_names) >= 2:
    # Get feature list for index lookup
    feature_list = X_train_scaled_df.columns.tolist()

    for feat1_name, feat2_name in combinations(top_features_pdp_names, 2):
        try:
            # === Fix: Convert feature names to integer indices ===
            feat1_idx = feature_list.index(feat1_name)
            feat2_idx = feature_list.index(feat2_name)

            features_to_plot = [(feat1_idx, feat2_idx)]

            pd_results = partial_dependence(loaded_model, X_train_scaled_df, features=features_to_plot,
                                            kind='average', grid_resolution=20)

            XX, YY = np.meshgrid(pd_results['values'][0], pd_results['values'][1])
            ZZ = pd_results['average'][0].T

            fig = plt.figure(figsize=(12, 9))
            ax = fig.add_subplot(111, projection='3d')
            surf = ax.plot_surface(XX, YY, ZZ, cmap='viridis', edgecolor='none', antialiased=True)
            fig.colorbar(surf, shrink=0.5, aspect=10, label='Partial Dependence Value')

            ax.set_xlabel(f'{feat1_name} (Standardized Value)', fontsize=10, labelpad=10)
            ax.set_ylabel(f'{feat2_name} (Standardized Value)', fontsize=10, labelpad=10)
            ax.set_zlabel('Dependence on Predicted Value (PDP)', fontsize=10, labelpad=10)
            ax.set_title(f'3D Partial Dependence Plot (3D PDP)\n{feat1_name} vs {feat2_name}', fontsize=14)
            ax.view_init(elev=20, azim=135)

            save_path = os.path.join(pdp_3d_save_dir, f'GBDT_PDP_3D_{feat1_name}_{feat2_name}.png')
            plt.savefig(save_path, dpi=300)
            plt.close(fig)
            print(f"Successfully plotted 3D PDP for {feat1_name} & {feat2_name}")
        except Exception as e:
            print(f"Error plotting 3D PDP for {feat1_name} & {feat2_name}: {e}")

In [ ]:
print('------------------------ Start SHAP Analysis ------------------------')
shap_save_dir = os.path.join(results_plot_save_dir, 'GBDT_SHAP_Plots_final')
os.makedirs(shap_save_dir, exist_ok=True)
explainer = shap.TreeExplainer(loaded_model)
shap_values = explainer(X_test_scaled_df)

# --- 3. SHAP Summary Bar Plot ---
print("\nDrawing SHAP Summary Plot (Bar Chart)...")
# Extract mean absolute values from shap_values as feature importance
shap_importance_vals = np.abs(shap_values.values).mean(axis=0)
shap_importance_df = pd.DataFrame({'Feature': X_test_scaled_df.columns, 'Importance': shap_importance_vals})
save_path_shap = os.path.join(shap_save_dir, 'GBDT_SHAP_Feature_Importance_Combined_final.png')
plot_importance_combined(shap_importance_df, 'SHAP Feature Importance (Mean Absolute SHAP Value)', save_path_shap, bar_color='#007bff')


print("Drawing SHAP Summary Plot (Scatter Distribution)...")
shap.summary_plot(shap_values, X_test_scaled_df, show=False)
plt.title('SHAP Feature Impact Overview (Scatter Distribution)', fontsize=16)
plt.tight_layout()
plt.savefig(os.path.join(shap_save_dir, 'GBDT_SHAP_summary_scatter.png'), dpi=300, bbox_inches='tight')
plt.close()

print("Drawing SHAP Dependence Plots...")
shap_dependence_save_dir = os.path.join(shap_save_dir, 'Dependence_Plots')
os.makedirs(shap_dependence_save_dir, exist_ok=True)
for feature_name in top_features_pdp_names:
    shap.dependence_plot(feature_name, shap_values.values, X_test_scaled_df, interaction_index="auto", show=False)
    plt.gcf().suptitle(f'SHAP Dependence Plot: {feature_name}', fontsize=16)
    plt.tight_layout()
    plt.savefig(os.path.join(shap_dependence_save_dir, f'GBDT_SHAP_dependence_{feature_name}.png'), dpi=300,
                bbox_inches='tight')
    plt.close()

# --- SHAP Waterfall Plot ---
print("Drawing SHAP Waterfall Plot (for the first test sample)...")
plt.figure()  # Create a new figure object
shap.plots.waterfall(shap_values[0], max_display=15, show=False)
plt.title('SHAP Waterfall Plot (Test Sample 0)', fontsize=16)
plt.tight_layout()
plt.savefig(os.path.join(shap_save_dir, 'GBDT_SHAP_waterfall_sample_0.png'), dpi=300, bbox_inches='tight')
plt.close()
print('---------------------------------------- SHAP Analysis Completed -----------------------------------------')

In [ ]:
print('---------------------------------------- Starting ALE Analysis -----------------------------------------')
ale_save_dir = os.path.join(results_plot_save_dir, 'GBDT_ALE_Plots_final')
os.makedirs(ale_save_dir, exist_ok=True)
print(f"ALE plots will be saved to: {ale_save_dir}")
top_features_ale_names = top_features_pdp_names

# --- 1D ALE Plotting (Colored) ---
print(f"\nStarting 1D ALE plots for the top {len(top_features_ale_names)} features...")
# Create a color list to assign different colors for each feature plot
colors = plt.cm.viridis(np.linspace(0, 0.85, len(top_features_ale_names)))

for i, feature_name in enumerate(top_features_ale_names):
    try:
        # Use PyALE to compute, which automatically creates a plot
        ale_eff = ale(X=X_train_scaled_df, model=loaded_model, feature=[feature_name],
                      feature_type='continuous', grid_size=50, include_CI=True, C=0.95)

        # Get current figure and axis
        fig, ax = plt.gcf(), plt.gca()
        current_color = colors[i]

        # Modify line color and width
        if ax.lines:
            ax.lines[0].set_color(current_color)
            ax.lines[0].set_linewidth(2.5)

        # Modify confidence interval fill color and transparency
        if ax.collections:
            ax.collections[0].set_facecolor(current_color)
            ax.collections[0].set_alpha(0.2)

        ax.set_title(f'Cumulative Local Effects (ALE) - Feature: {feature_name}', fontsize=16)
        ax.set_xlabel(f'{feature_name} (standardized value)', fontsize=12)
        ax.set_ylabel('ALE (effect on prediction)', fontsize=12)
        plt.tight_layout()
        plt.savefig(os.path.join(ale_save_dir, f'GBDT_ALE_1D_{feature_name}.png'), dpi=300, bbox_inches='tight')
        plt.close(fig)
    except Exception as e:
        print(f"Error plotting 1D ALE for {feature_name}: {e}")
        if plt.get_fignums():
            plt.close('all')

# --- 2D ALE Plotting (Colored Heatmap) ---
print(f"\nStarting 2D ALE plots for the most important feature pairs...")
if len(top_features_ale_names) >= 2:
    for feat1_name, feat2_name in combinations(top_features_ale_names, 2):
        try:
            # Compute ALE effect only, do not plot automatically
            ale_eff_2d = ale(X=X_train_scaled_df, model=loaded_model, feature=[feat1_name, feat2_name],
                             feature_type='continuous', grid_size=30, plot=False)

            # Manually create a colored heatmap
            fig, ax = plt.subplots(figsize=(8, 7))
            im = ax.pcolormesh(ale_eff_2d.index, ale_eff_2d.columns, ale_eff_2d.values.T, cmap='viridis',
                               shading='auto')
            fig.colorbar(im, ax=ax, label='ALE (effect on prediction)')

            ax.set_title(f'2D ALE: {feat1_name} vs {feat2_name}', fontsize=16)
            ax.set_xlabel(f'{feat1_name} (standardized value)', fontsize=12)
            ax.set_ylabel(f'{feat2_name} (standardized value)', fontsize=12)
            plt.tight_layout()
            plt.savefig(os.path.join(ale_save_dir, f'GBDT_ALE_2D_{feat1_name}_vs_{feat2_name}.png'), dpi=300,
                        bbox_inches='tight')
            plt.close(fig)
        except Exception as e:
            print(f"Error plotting 2D ALE for {feat1_name} & {feat2_name}: {e}")
            if plt.get_fignums():
                plt.close('all')

print('---------------------------------------- ALE Analysis Completed -----------------------------------------')
print('---------------------------------------- Script Execution Finished -----------------------------------------')